## LM Studio - easy inference

### Requirements
- LM Studio
- LM Studio's server running on "http://localhost:1234/v1" or any other easy accessible address

We aim to obtain the result in JSON format, which is a common data format with diverse uses in data handling. The expected output should look like this:

```json
{
    "is_fraudulent": true
}
```

```json
{
    "is_fraudulent": false
}
```

The language model categorizes the sms as fraudulent or not.

This is done by a combination of:
- Prompt engineering
- Fine-tuning
- Hyperparameter optimization
- Function-calling or JSON outputs

### Base Implementation

In [1]:
from openai import OpenAI

local_server = "http://localhost:1234/v1"
client = OpenAI(base_url=local_server, api_key="sk_1234567890")

# Prompts:

CONDIFDENCE_SYSTEM_MESSAGE = """You are excellent at providing responses in JSON format.

You will be given "Messages" and your job is to predict if a message is fraudulent or not.

You only respond following this json schema:

{
    "name": "predict_fraudulence",
    "description": "Assess if the given message is likely to be fraudulent.",
    "response": {
        "is_fraudulent": {
            "type": "boolean",
            "description": "Whether the message is predicted to be fraudulent."
        },
        "confidence": {
            "type": "number",
            "description": "The confidence score of the prediction, ranging from 0.0 to 1.0, where 1.0 represents full confidence."
        }
    }
}
"""

BOOL_SYSTEM_MESSAGE = """You are excellent at providing responses in JSON format.

You will be given "Messages" and your job is to predict if a message is fraudulent or not.

You only respond following this json schema:

{
    "is_fraudulent": {
        "type": "boolean",
        "description": "Whether the message is predicted to be fraudulent."
    }
}

JSON OUTPUT ONLY!!!
"""

# Run inferenece on API - LM Studio (mimics OpenAI API)

def predict_fraudulence(user_query):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": BOOL_SYSTEM_MESSAGE},
            {"role": "user", "content": user_query},
        ],
    )
    return response.choices[0].message.content

### Run inference and get JSON output

In [3]:
# Run prediction
user_query = "I am a Nigerian prince and I want to give you $1,000,000"
prediction = predict_fraudulence(user_query)


In [ ]:
print(prediction)

{
    "is_fraudulent": true
}


In [ ]:
# Extract the JSON from the response
import json
prediction_json = json.loads(prediction)


In [ ]:
prediction_json["is_fraudulent"]

True

### Full process as a function

Run inference, get JSON output, and save the result result (with error handling).

In [ ]:
import json
from openai import OpenAI
from typing import Optional

local_server = "http://localhost:1234/v1"
client = OpenAI(base_url=local_server, api_key="sk_1234567890")

# Choose which system message to use based on whether you want a confidence score
system_message = BOOL_SYSTEM_MESSAGE
def predict_fraudulence(user_query):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_query},
            ]
        )
        prediction_content = response.choices[0].message.content
        prediction_json = json.loads(prediction_content)
        
        # Validate the prediction JSON
        if "is_fraudulent" in prediction_json and \
            (type(prediction_json["is_fraudulent"]) == bool):
            return prediction_json
        else:
            raise ValueError("Invalid JSON structure from prediction: {}".format(prediction_content))
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [ ]:
# Run prediction
user_query = "Hi Chris! Can you send me $10? I need it for a school project. Thanks!"
prediction_json = predict_fraudulence(user_query)

if prediction_json:
    print(prediction_json)
else:
    print("Failed to get a valid prediction.")

{'is_fraudulent': True}
